### Predicting with Bayesian Neural Network Classifier
Binary Case

In [1]:
import rasterio

from eis_toolkit.prediction.bayesian_neural_network import bayesian_neural_network_classifier_train
from eis_toolkit.prediction.machine_learning_general import prepare_data_for_ml, reshape_predictions
from eis_toolkit.evaluation.scoring import score_predictions
from eis_toolkit.evaluation.classification_probability_evaluation import summarize_probability_metrics

Load data

In [ ]:
# Put in your own data to test the workflow
train_features_paths = [
    "../tests/data/local/data/train_feature_1.tif",
    "../tests/data/local/data/train_feature_2.tif",
    "...",
]
train_labels_path = "../tests/data/local/data/train_labels.tif"

test_features_paths = [
    "../tests/data/local/data/test_feature_1.tif",
    "../tests/data/local/data/test_feature_2.tif",
    "...",
]
test_labels_path = "../tests/data/local/data/test_labels.tif"

Prepare data for modeling (based on **rasterized** data)

In [ ]:
X_train, y_train, _, _ = prepare_data_for_ml(test_features_paths, test_labels_path)
X_test, y_test, reference_meta, nodata_mask = prepare_data_for_ml(test_features_paths, test_labels_path)


Run Bayesian classifier

In [ ]:
model = bayesian_neural_network_classifier_train(X_train, y_train)

model_history = model.history()
print(model_history.history)


Predict **with uncertainty**

In [ ]:
probabilities, uncertainties = model.predict_with_uncertainty(X_test)


Reshape into raster dimensions

In [ ]:
reshaped_predictions = reshape_predictions(
    probabilities,
    reference_meta["height"],
    reference_meta["width"],
    nodata_mask
)

reshaped_uncertainties = reshape_predictions(
    uncertainties,
    reference_meta["height"],
    reference_meta["width"],
    nodata_mask
)


Get some scores

In [ ]:
metrics = score_predictions(
    y_true=y_train,
    y_pred=probabilities > 0.5,
    metrics=["accuracy", "precision", "recall", "f1"],
    decimals=3
)

print(metrics)

prob_metrics = summarize_probability_metrics(
    y_true=y_train,
    y_prob=probabilities,
    decimals=3,
)

print(prob_metrics)


Save results

In [ ]:
with rasterio.open(
    "file_path_predictions.tif", "w", **reference_meta) as raster:
    raster.write(reshaped_predictions, 1)
    raster.close()

with rasterio.open(
    "file_path_uncertainties.tif", "w", **reference_meta) as raster:
    raster.write(reshaped_uncertainties, 1)
    raster.close()